In [2]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader,Dataset
from torchsummary import summary


In [3]:
file_path_1='./samples/training.pkl'
file_path_2='./samples/testing.pkl'

In [4]:
class CombinedCustomDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'rb') as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features, label = self.data[idx]
        features = torch.FloatTensor(features)
        label = torch.tensor(label, dtype=torch.float32)
        return features, label

In [5]:
train_dataset = CombinedCustomDataset(file_path_1)
test_dataset = CombinedCustomDataset(file_path_2)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [6]:
class Train_Model(nn.Module):
    def __init__(self, in_channel=2, input_length=6250):
        super(Train_Model,self).__init__()
        self.c1=nn.Conv1d(in_channel,out_channels=5,kernel_size=10,stride=1)
        self.b1=nn.BatchNorm1d(5)
        self.p1=nn.MaxPool1d(kernel_size=2,stride=2)
        self.c2=nn.Conv1d(in_channels=5,out_channels=10,kernel_size=10,stride=1)
        self.p2=nn.MaxPool1d(kernel_size=2,stride=2)
        self.c3 = nn.Conv1d(10, 10, kernel_size=10, stride=1)
        self.p3 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.c4 = nn.Conv1d(10, 10, kernel_size=10, stride=1)
        self.p4 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.c5 = nn.Conv1d(10, 15, kernel_size=10, stride=1)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.5)

        self.flatten=nn.Flatten()


        self.fc1=nn.Linear(15,10)
        self.fc2=nn.Linear(10,5)
        self.output=nn.Linear(5,1)

    def forward(self,x):
        x=self.c1(x)
        x=self.b1(x)
        x=nn.LeakyReLU()(x)
        x = self.p1(x)
        
        x = self.c2(x)
        x = nn.LeakyReLU()(x)
        x = self.p2(x)
        
        x = self.c3(x)
        x = nn.LeakyReLU()(x)
        x = self.p3(x)
        
        x = self.c4(x)
        x = nn.LeakyReLU()(x)
        x = self.p4(x)
        
        x = self.c5(x)
        x = nn.LeakyReLU()(x)
        x = self.global_avg_pool(x)
        
        x = self.flatten(x)
        x = self.dropout(x)
        
        x = self.fc1(x)
        x = nn.LeakyReLU()(x)
        x = self.fc2(x)
        x = nn.LeakyReLU()(x)
        
        x = torch.sigmoid(self.output(x))
        return x


model=Train_Model(in_channel=2,input_length=7500)
criterian=nn.BCELoss()
optimizers=torch.optim.Adam(model.parameters(),lr=0.001)


In [7]:
model=Train_Model(in_channel=2,input_length=6250)
summary(model, (2, 6250)) 


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 5, 6241]             105
       BatchNorm1d-2              [-1, 5, 6241]              10
         MaxPool1d-3              [-1, 5, 3120]               0
            Conv1d-4             [-1, 10, 3111]             510
         MaxPool1d-5             [-1, 10, 1555]               0
            Conv1d-6             [-1, 10, 1546]           1,010
         AvgPool1d-7              [-1, 10, 773]               0
            Conv1d-8              [-1, 10, 764]           1,010
         AvgPool1d-9              [-1, 10, 382]               0
           Conv1d-10              [-1, 15, 373]           1,515
AdaptiveAvgPool1d-11                [-1, 15, 1]               0
          Flatten-12                   [-1, 15]               0
          Dropout-13                   [-1, 15]               0
           Linear-14                   

In [8]:
model.train()
epochs=30
losses=[]
for epoch in range(epochs):
    for inputs,outputs in train_loader:
        y_pred=model.forward(inputs).squeeze()
        loss=criterian(y_pred,outputs)
        losses.append(loss)
        optimizers.zero_grad()
        loss.backward()
        optimizers.step()

    print('epoch {}/{}:  Loss: {}'.format(epoch,epochs,loss.item()))

epoch 0/70:  Loss: 0.694423496723175
epoch 1/70:  Loss: 0.6884756088256836
epoch 2/70:  Loss: 0.6925452947616577
epoch 3/70:  Loss: 0.6909406781196594
epoch 4/70:  Loss: 0.6948099732398987
epoch 5/70:  Loss: 0.6933664083480835
epoch 6/70:  Loss: 0.6896024346351624
epoch 7/70:  Loss: 0.6932126879692078
epoch 8/70:  Loss: 0.6932287812232971
epoch 9/70:  Loss: 0.6938400268554688
epoch 10/70:  Loss: 0.6870933771133423
epoch 11/70:  Loss: 0.6926371455192566
epoch 12/70:  Loss: 0.6949718594551086
epoch 13/70:  Loss: 0.6915622353553772
epoch 14/70:  Loss: 0.6952152252197266
epoch 15/70:  Loss: 0.6901819109916687
epoch 16/70:  Loss: 0.6952898502349854
epoch 17/70:  Loss: 0.6975070238113403
epoch 18/70:  Loss: 0.7014557123184204
epoch 19/70:  Loss: 0.6951996684074402
epoch 20/70:  Loss: 0.6935819387435913
epoch 21/70:  Loss: 0.694301426410675
epoch 22/70:  Loss: 0.6894053816795349
epoch 23/70:  Loss: 0.6975651383399963
epoch 24/70:  Loss: 0.6952767968177795
epoch 25/70:  Loss: 0.700975954532623

In [11]:
model.eval()
total_accuracy=0

with torch.no_grad():
    for inputs, targets in test_loader:
        test_preds = model(inputs).squeeze()  # Squeeze to ensure dimension match
        test_loss = criterian(test_preds, targets)
        
        # Calculate accuracy
        predicted = test_preds.round()  # Assuming threshold of 0.5
        accuracy = (predicted == targets).float().mean().item()
        total_accuracy += accuracy

    avg_accuracy = total_accuracy / len(test_loader)
    print(f'Test Loss: {test_loss.item()}, Average Accuracy: {avg_accuracy}')



Test Loss: 0.7244420051574707, Average Accuracy: 0.49857954545454547


In [10]:
torch.save(model, 'Best_Model.pth')